# Singler-User Implementation

This notebook implements single-user CCS-AMP encoder/decoder without power control.

In [1]:
import numpy as np
import FactorGraphGeneration as FGG

# OuterCode1 = FGG.Triadic8(16)
# OuterCode1 = FGG.CCSDS_ldpc_n32_k16(8)
# OuterCode1 = FGG.MacKay_96_33_964(8)
# OuterCode1 = FGG.WRAN_N384_K192(8)
OuterCode1 = FGG.WIMAX_768_640(9)

Size of parity check matrix: (128, 768)
Rank of parity check matrix: 128
[[1 0 0 ... 0 1 0]
 [0 1 0 ... 0 0 1]
 [0 0 1 ... 0 1 0]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]]
Number of parity column indices: 128
Number of parity nodes: 128
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
Number of information nodes: 640
[[1 0 0 ... 0 1 0]
 [0 1 1 ... 0 0 1]
 [1 0 0 ... 0 1 0]
 ...
 [0 0 1 ... 1 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]]


## Fast Hadamard Transforms

In [2]:
from pyfht import block_sub_fht

## SPARC Codebook

We use the `block_sub_fht` which computes the equivalent of $A.\beta$ by using $L$ separate $M\times M$ Hadamard matrices. However we want each entry to be divided by $\sqrt{n}$ to get the right variance, and we need to do a reshape on the output to get column vectors, so we'll wrap those operations here.

Returns two functions `Ab` and `Az` which compute $A\cdot B$ and $z^T\cdot A$ respectively.

In [3]:
def sparc_codebook(L, M, n,P):
    Ax, Ay, _ = block_sub_fht(n, M, L, seed=None, ordering=None) # seed must be explicit
    def Ab(b):
        return Ax(b).reshape(-1, 1)/ np.sqrt(n)
    def Az(z):
        return Ay(z).reshape(-1, 1)/ np.sqrt(n) 
    return Ab, Az

# Vector Approximation

This function outputs the closest approximation to the input vector given that its L1 norm is 1 and no entry is greater than 1/K

In [4]:
def approximateVector(x, K):    

    # normalize initial value of x
    xOrig = x / np.linalg.norm(x, ord=1)
    
    # create vector to hold best approximation of x
    xHt = xOrig.copy()
    u = np.zeros(len(xHt))
    
    # run approximation algorithm
    while np.amax(xHt) > (1/K):
        minIndices = np.argmin([(1/K)*np.ones(xHt.shape), xHt], axis=0)
        xHt = np.min([(1/K)*np.ones(xHt.shape), xHt], axis=0)
        
        deficit = 1 - np.linalg.norm(xHt, ord=1)
        
        if deficit > 0:
            mIxHtNorm = np.linalg.norm((xHt*minIndices), ord=1)
            scaleFactor = (deficit + mIxHtNorm) / mIxHtNorm
            xHt = scaleFactor*(minIndices*xHt) + (1/K)*(np.ones(xHt.shape) - minIndices)

    # return admissible approximation of x
    return xHt

## Posterior Mean Estimator (PME)

This function implements the posterior mean estimator for situations where prior probabilities are uninformative.

In [5]:
def pme0(q, r, d, tau):
    """Posterior mean estimator (PME)
    
    Args:
        q (float): Prior probability
        r (float): Effective observation
        d (float): Signal amplitude
        tau (float): Standard deviation of noise
    Returns:
        sHat (float): Probability s is one
    
    """
    sHat = ( q*np.exp( -(r-d)**2 / (2*(tau**2)) ) \
            / ( q*np.exp( -(r-d)**2 / (2*(tau**2))) + (1-q)*np.exp( -r**2 / (2*(tau**2))) ) ).astype(float)
    return sHat

In [6]:
def pme1(q, r, d, tau):
    """Posterior mean estimator (PME)
    
    Args:
        q (float): Prior probability
        r (float): Effective observation
        Pl (float): Signal amplitudes
        tau (float): Standard deviation of noise
    Returns:
        sHat (float): Probability s is one
    
    """
    exps = q * np.exp(r * d / tau**2)
    sums = np.sum(exps,axis=1)
    sHat = np.zeros(r.shape)
    for idx in range(len(sums)):
        sHat[idx,:] = (d * exps[idx, :] / sums[idx])
    
    return np.reshape(sHat, (-1, 1))

# Dynamic Denoiser

This function performs believe propagation (BP) on the factor graph of the outer code.

In [7]:
def dynamicDenoiser(r,OuterCode,K,tau,d,numBPiter):
    """
    Args:
        r (float): Effective observation
        d (float): Signal amplitude
        tau (float): Standard deviation of noise
    """
    M = OuterCode.sparseseclength
    L = OuterCode.varcount

    p0 = 1-(1-1/M)**K
    p1 = p0*np.ones(r.shape, dtype=float)
    mu = np.zeros(r.shape, dtype=float)

    # Compute local estimate (lambda) based on effective observation using PME.
    localEstimates = pme0(p0, r, d, tau)
    
    # Reshape local estimate (lambda) into an LxM matrix
    Beta = localEstimates.reshape(L,-1)
    OuterCode.reset()
    for varnodeid in OuterCode.varlist:
        idx = varnodeid - 1
        Beta[idx,:] = approximateVector(Beta[idx,:], K)
        OuterCode.setobservation(varnodeid, Beta[idx,:])
    
    for iteration in range(numBPiter):
        OuterCode.updatechecks()
        OuterCode.updatevars()

    for varnodeid in OuterCode.varlist:
        idx = varnodeid - 1
        # Beta[idx,:] = OuterCode.getestimate(varnodeid)
        Beta[idx,:] = OuterCode.getextrinsicestimate(varnodeid)
        mu[idx*M:(idx+1)*M] = approximateVector(Beta[idx,:], K).reshape(-1,1)

    return mu

## AMP
This is the actual AMP algorithm. It's a mostly straightforward transcription from the relevant equations, but note we use `longdouble` types because the expentials are often too big to fit into a normal `double`.

In [8]:
def amp_state_update(z, s, d, Ab, Az, K, numBPiter, OuterCode):

    """
    Args:
        s: State update through AMP composite iteration
        z: Residual update through AMP composite iteration
        tau (float): Standard deviation of noise
        mu: Product of messages from adjoining factors
    """
    n = z.size

    # Compute tau online using the residual
    tau = np.sqrt(np.sum(z**2)/n)

    # Compute effective observation
    r = (d*s + Az(z))
    # Compute updated state
    mu = dynamicDenoiser(r, OuterCode, K, tau, d, numBPiter)
#     s = pme0(mu, r, d, tau)
    s = pme1(np.reshape(mu, (L,-1)), np.reshape(r, (L, -1)), d, tau)
        
    return s

In [9]:
def amp_residual(y, z, s, d, Ab):
    """
    Args:
        s1: State update through AMP composite iteration
        s2: State update through AMP composite iteration
        y: Original observation
        tau (float): Standard deviation of noise
    """
    n = y.size
    
    # Compute tau online using the residual
    tau = np.sqrt(np.sum(z**2)/n)

    # Compute residual
    Onsager = (d**2)*(np.sum(s) - np.sum(s**2))
    z_plus = y - d*Ab(s) + (z/(n*tau**2))*Onsager
    
    return z_plus

# Simulation

In [ ]:
K = 1 # Number of active users

# OuterCode1 = FGG.Graph64(8)
# n = 32768 # Total number of channel uses (real d.o.f)

# OuterCode1 = FGG.MacKay_96_33_964(8)
# n = 38400 # Total number of channel uses (real d.o.f)

# OuterCode1 = FGG.WRAN_N384_K192(8)
# n = 98304 # Total number of channel uses (real d.o.f)

# OuterCode1 = FGG.WIMAX_768_640(9)
n = 7200 # Total number of channel uses (real d.o.f)


B = OuterCode1.infocount * OuterCode1.seclength # Payload size of each active user
L = OuterCode1.varcount # Number of sections/sub-blocks
T = 40 # Number of AMP iterations
J = OuterCode1.seclength # Length of each coded sub-block
M = OuterCode1.sparseseclength # Length of each section


numBPiter = 1; # Number of BP iterations on outer code. 1 seems to be good enough & AMP theory including state evolution valid only for one BP iteration
EbNodB = 10.0 # Energy per bit. With iterative extension, operating EbN0 falls to 2.05 dB for 25 users with 1 round SIC
simCount = 1 # number of simulations

# EbN0 in linear scale
EbNo = 10**(EbNodB/10)
P = 2*B*EbNo/n
σ_n = 1

# We assume equal power allocation for all the sections. Code has to be modified a little to accomodate non-uniform power allocations
Phat = n*P/L
d = np.sqrt(n*P/L)

msgDetected=0

for simIndex in range(simCount):
    print('Simulation Number: ' + str(simIndex))
    
    # Generate active users message sequences
    messages = np.random.randint(2, size=(K, B))

    # Outer-encode the message sequences
    codewords = OuterCode1.encodemessages(messages)
    for codeword in codewords:
        OuterCode1.testvalid(codeword)
    
    # Convert indices to sparse representation
    # sTrue: True state
    sTrue = np.sum(codewords, axis=0) #np.sum(codewords, axis=0)

    # Generate the binned SPARC codebook
    Ab, Az = sparc_codebook(L, M, n, P)
    
    # Generate our transmitted signal X
    x = np.sqrt(Phat)*Ab(sTrue)
    
    # Generate random channel noise and thus also received signal y
    noise = np.random.randn(n, 1) * σ_n
    y = (x + noise)

    z = y.copy()
    s = np.zeros((L*M, 1)) # No interference in dexing code below

    for t in range(T):
        s = amp_state_update(z, s, d, Ab, Az, K, numBPiter, OuterCode1)
        z = amp_residual(y, z, s, d, Ab)

    s = sTrue
    
    print('Graph Decode')
    print(codewords.shape)

    # Decoding with Graph
    originallist = codewords.copy()
    recoveredcodewords = OuterCode1.decoder(codewords, 10)

    # Calculation of per-user prob err
    simMsgDetected = 0
    matches = FGG.numbermatches(originallist, recoveredcodewords)
    
    print('Outcome: ' + str(matches) + ' out of ' + str(K))
    msgDetected = msgDetected + matches
    
errorRate= (K*simCount - msgDetected)/(K*simCount)
print("Per user probability of error = ", errorRate)


Simulation Number: 0
Graph Decode
(1, 393216)
Root section ID: 166
Root section ID: 175
Root section ID: 174
Root section ID: 169
Root section ID: 170
Root section ID: 171
Root section ID: 172
Root section ID: 124
Root section ID: 173
Root section ID: 511
